In [25]:
import json
import requests
import pandas as pd
import datetime
import psycopg2

URL = 'http://127.0.0.1:5000'
requests.request('GET', URL + '/itens').status_code

itens_usage = requests.request('GET', URL + '/itens').json()['usage']
df=pd.DataFrame(itens_usage)
df_catalizador = df[df['name']=='01CATALIZADOR'].to_json(orient='values')

hostname='localhost'
database='BFC_data'
username='postgres'
password='MyPostgreSQL'
port_id=5432
conn = None
cur=None

try:
    conn = psycopg2.connect(
                            host=hostname,
                            dbname=database,
                            user=username,
                            password=password,
                            port=port_id)
    cur = conn.cursor()
    
    #Verificando a existência da tabela
    cur.execute("select * from information_schema.tables where table_name=%s", ('vw_01catalizador',))
    if bool(cur.rowcount): #Se a tabela existir
        
        catalizador_list = list(json.loads(df_catalizador)) #Lista que vem da API
        
        #Atualizando itens antigos
        for itens in catalizador_list:
            
            cur.execute('UPDATE vw_01catalizador SET name = %s, data = %s, value = %s WHERE item_id = %s',
                        (itens[1],itens[2],itens[3],itens[0]))
                        #Este símbolo '%s' é conhecido como place holder. Os valores dos 's' devem ser colocados na ordem
            conn.commit()
        
        #Inserindo os dados novos
        lista_atual = []
    
        cur.execute('SELECT * FROM vw_01catalizador')
        for linha in cur.fetchall():
            lista_atual.append(list(linha)) #Lista que está no PostgreSQL

        conn.commit()
    
        novos_itens = [] #Lista onde serão colocados somente os itens que não estiverem no PostgreSQL
    
        for itens in catalizador_list:
            if not itens in lista_atual:
                novos_itens.append(itens)
        
        if len(novos_itens)==0:
            
            print('Não há itens para adicionar')
    
        else:
        
            insert_script = 'INSERT INTO vw_01catalizador(item_id, name, data, value) VALUES(%s,%s,%s,%s)'

            catalizador_tuple = tuple(novos_itens) #Tupla somente com os dados novos

            for linha in catalizador_tuple:

                cur.execute(insert_script,linha)
                conn.commit()
            
            print('Novos dados inseridos com sucesso')
            print(novos_itens)
            
    else: #Caso a tabela ainda não exista   
    
        create_script = '''CREATE TABLE IF NOT EXISTS vw_01catalizador(
                            item_id    INT PRIMARY KEY,
                            name       VARCHAR(20),
                            data       VARCHAR(20),
                            value      INT)'''
    
        cur.execute(create_script)
    
        insert_script = 'INSERT INTO vw_01catalizador(item_id, name, data, value) VALUES(%s,%s,%s,%s)'
    
        catalizador_tuple = tuple(json.loads(df_catalizador)) #Tupla com todos os dados vindos da API
    
        for linha in catalizador_tuple:
        
            cur.execute(insert_script,linha)
        
        cur.execute('SELECT * FROM vw_01catalizador')
        for linha in cur.fetchall():
            print(linha)
    
        conn.commit()
    
except Exception as error:
    print(error)
    
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Não há itens para adicionar
